In [1]:
# This notebook is used to read in model data from MOM6
# and extract the fields needed for making figures 2, 3, 15, and 16 in
# Reichl et al., 2024 (submitted to AGU's ESS journal)

import xarray as xr


In [2]:
data_loc = "/archive/"

Model=xr.Dataset()
for TAG,LBL in [['OM4p25_JRA55do1.5_0netfw_cycle1','OM4'],
                ['OM4p25_JRA55do1.5_0netfw_MStarCap1p25_cycle1','OM4-ePBLcap'],
                ['OM4p25_JRA55do1.5_0netfw_KV0_cycle1','OM4-KV0'],
                ['OM4p25_JRA55do1.5_0netfw_KV0-KSBBLup-MStarCap1p25_cycle1','OM4up'],
                ['CM4_historical','CM4'],
               ]:

    if LBL[:3]=='OM4':
        YEARS=['2003-2007','2008-2012','2013-2017']
        LC=data_loc+'/bgr/FMS2021.02_mom6_devgfdl_20221223/'
        VZ='gfdl.ncrc5-intel22-prod'
    else:
        LC=data_loc+'/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/'
        VZ='gfdl.ncrc4-intel16-prod-openmp'
        YEARS=['2000-2004','2005-2009','2010-2014',]
    pth = 'pp/ocean_annual_z_1x1deg/av/annual_5yr/'

    Zdir  = (LC+'/'+TAG+'/'+VZ+'/'+pth)


    files_z=[]
    for YRS in YEARS:
        files_z+=[Zdir+'ocean_annual_z_1x1deg.'+YRS+'.ann.nc']

    lst=''
    for ii in files_z:
        lst+=ii+' '

    print(lst)

    FZ = xr.open_mfdataset(files_z)
    FZ = FZ.assign_coords(lon=(FZ.lon.values - 20)%360.+20)
    FZ = FZ.roll(lon=-20, roll_coords=True)


    LON = [20.,380.]
    LAT= [-1.,1.]
    DMax=500

    T_Model = FZ.thetao.sel(lat=slice(LAT[0],LAT[1]),
                        lon=slice(LON[0],LON[1])
                       ).mean(dim=['time','lat']).sel(z_l=slice(0,DMax))
    S_Model = FZ.so.sel(lat=slice(LAT[0],LAT[1]),
                        lon=slice(LON[0],LON[1])
                       ).mean(dim=['time','lat']).sel(z_l=slice(0,DMax))
    Model = xr.merge([Model,T_Model])
    Model = xr.merge([Model,S_Model])
    Model = Model.rename({'thetao':'T_'+LBL,'so':'S_'+LBL})
    

/archive//bgr/FMS2021.02_mom6_devgfdl_20221223//OM4p25_JRA55do1.5_0netfw_cycle1/gfdl.ncrc5-intel22-prod/pp/ocean_annual_z_1x1deg/av/annual_5yr/ocean_annual_z_1x1deg.2003-2007.ann.nc /archive//bgr/FMS2021.02_mom6_devgfdl_20221223//OM4p25_JRA55do1.5_0netfw_cycle1/gfdl.ncrc5-intel22-prod/pp/ocean_annual_z_1x1deg/av/annual_5yr/ocean_annual_z_1x1deg.2008-2012.ann.nc /archive//bgr/FMS2021.02_mom6_devgfdl_20221223//OM4p25_JRA55do1.5_0netfw_cycle1/gfdl.ncrc5-intel22-prod/pp/ocean_annual_z_1x1deg/av/annual_5yr/ocean_annual_z_1x1deg.2013-2017.ann.nc 
/archive//bgr/FMS2021.02_mom6_devgfdl_20221223//OM4p25_JRA55do1.5_0netfw_MStarCap1p25_cycle1/gfdl.ncrc5-intel22-prod/pp/ocean_annual_z_1x1deg/av/annual_5yr/ocean_annual_z_1x1deg.2003-2007.ann.nc /archive//bgr/FMS2021.02_mom6_devgfdl_20221223//OM4p25_JRA55do1.5_0netfw_MStarCap1p25_cycle1/gfdl.ncrc5-intel22-prod/pp/ocean_annual_z_1x1deg/av/annual_5yr/ocean_annual_z_1x1deg.2008-2012.ann.nc /archive//bgr/FMS2021.02_mom6_devgfdl_20221223//OM4p25_JRA55do1

In [3]:
Model

<xarray.Dataset>
Dimensions:        (z_l: 14, lon: 360)
Coordinates:
  * z_l            (z_l) float64 2.5 10.0 20.0 32.5 ... 250.0 312.5 400.0 500.0
  * lon            (lon) float64 20.5 21.5 22.5 23.5 ... 376.5 377.5 378.5 379.5
Data variables:
    T_OM4          (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    S_OM4          (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    T_OM4-ePBLcap  (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    S_OM4-ePBLcap  (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    T_OM4-KV0      (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    S_OM4-KV0      (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    T_OM4up        (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    S_OM4up        (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    T_CM4          (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>
    S_CM4          (z_l, lon) float32 dask.array<chunksize=(14, 360), meta=np.ndarray>

In [4]:
Model.to_netcdf('../Data/Model_Outputs.nc')

/net2/bgr/anaconda3/envs/oceans/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
